# ODTC Door Command Test

This notebook demonstrates connecting to an ODTC device and testing the open/close door commands.

## Setup

Before running, ensure:
1. The ODTC device is powered on and connected to the network
2. You know the IP address of the ODTC device
3. Your computer is on the same network as the ODTC
4. The ODTC device is accessible (not locked by another client)

# 1. Libraries and configuration

In [1]:
# Import required modules
import logging

from pylabrobot.thermocycling.inheco import InhecoODTC, ODTCBackend

# Configure logging to show debug information
logging.basicConfig(
    level=logging.DEBUG,  # Set to DEBUG for more verbose output
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)

# Get logger for ODTC backend to see status debugging
odtc_logger = logging.getLogger('pylabrobot.thermocycling.inheco.odtc_backend')
odtc_logger.setLevel(logging.DEBUG)  # Set to DEBUG for more verbose output

sila_logger = logging.getLogger('pylabrobot.thermocycling.inheco.odtc_sila_interface')
sila_logger.setLevel(logging.DEBUG)  # Set to DEBUG for more verbose output

print("Logging configured. Debug information will be displayed.")

Logging configured. Debug information will be displayed.


In [2]:
# Configuration
backend = ODTCBackend(odtc_ip="192.168.1.50", logger=odtc_logger)
tc = InhecoODTC(name="odtc_test", backend=backend, model="96")
print("Backend and thermocycler objects created.")
print(f"Backend logger level: {backend.logger.level} ({logging.getLevelName(backend.logger.level)})")

Backend and thermocycler objects created.
Backend logger level: 10 (DEBUG)


# 2. Connection and method introspection

In [3]:
print("Starting device setup...")
print("=" * 60)

await tc.setup()

print("=" * 60)
print("✓ Connected and initialized successfully!")

Starting device setup...


2026-01-26 16:00:53,874 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - Device reset (unlocked)
2026-01-26 16:00:54,149 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - GetStatus returned raw state: 'standby' (type: str)
2026-01-26 16:00:54,149 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - Device is in standby state, calling Initialize...
2026-01-26 16:01:02,209 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - Device successfully initialized and is in idle state


✓ Connected and initialized successfully!


In [4]:
# Get methods and premethods stored on the device
# Note: Requires device to be in Idle state (after setup/initialize)
# Returns ODTCMethodSet object with premethods and methods attributes
method_set = await tc.get_method_set()

print(f"PreMethods ({len(method_set.premethods)}):")
if method_set.premethods:
    for pm in method_set.premethods:
        print(f"  - {pm.name}")
else:
    print("  (none)")

print(f"\nMethods ({len(method_set.methods)}):")
if method_set.methods:
    for m in method_set.methods:
        print(f"  - {m.name}")
else:
    print("  (none)")

# You can also get a specific method by name:
# method = await tc.get_method_by_name("PCR_30cycles")

PreMethods (15):
  - PRE25
  - PRE25LID50
  - PRE55
  - PREDT
  - Pre_25
  - Pre25
  - Pre_25_test
  - Pre_60
  - Pre_4
  - dude
  - START
  - EVOPLUS_Init_4C
  - EVOPLUS_Init_110C
  - EVOPLUS_Init_Block20CLid85C
  - EVOPLUS_Init_Block20CLid40C

Methods (61):
  - M18_Abnahmetest
  - M23_LEAK
  - M24_LEAKCYCLE
  - M22_A-RUNIN
  - M22_B-RUNIN
  - M30_PCULOAD
  - M33_Abnahmetest
  - M34_Dauertest
  - M35_VCLE
  - M36_Verifikation
  - M37_BGI-Kon
  - M39_RMATEST
  - M18_Abnahmetest_384
  - M23_LEAK_384
  - M22_A-RUNIN_384
  - M22_B-RUNIN_384
  - M30_PCULOAD_384
  - M33_Abnahmetest_384
  - M34_Dauertest_384
  - M35_VCLE_384
  - M36_Verifikation_384
  - M37_BGI-Kon_384
  - M39_RMATEST_384
  - M120_OVT
  - M320_OVT
  - M121_OVT
  - M321_OVT
  - M123_OVT
  - M323_OVT
  - M124_OVT
  - M324_OVT
  - M125_OVT
  - M325_OVT
  - PMA cycle
  - Test
  - DC4_ProK_digestion
  - DC4_3Prime_Ligation
  - DC4_ProK_digestion_1_test
  - DC4_3Prime_Ligation_test
  - DC4_5Prime_Ligation_test
  - DC4_USER_Ligatio

# 3. Commands

### Non-Blocking Door Operations

The door commands support non-blocking execution using the `wait=False` parameter. This returns a `CommandExecution` handle that you can await later, allowing you to do other work while the door operation completes. 

Note: Just set wait=True to treat these as blocking and run them as normal commands

In [5]:
# Non-blocking door close
print("Starting door close (non-blocking)...")
door_closing = await tc.close_door(wait=False)
print(f"✓ Door close command started! Request ID: {door_closing.request_id}")

Starting door close (non-blocking)...
✓ Door close command started! Request ID: 613788033


In [6]:
# Access DataEvents for a command execution
# (Note: DataEvents are primarily used for method execution, but the pattern works for all commands)
# You can also use the wait() method explicitly
print("Waiting for door to close...")
await door_closing.wait()
print("✓ Door close completed!")

door_opening = await tc.open_door(wait=False)
print(f"Door opening with request ID: {door_opening.request_id}")

# Get DataEvents for this command (if any were collected)
events = await door_opening.get_data_events()
print(f"DataEvents collected: {len(events)}")

# Wait for completion
await door_opening
print("✓ Door open completed!")

Waiting for door to close...
✓ Door close completed!
Door opening with request ID: 1835317567
DataEvents collected: 0
✓ Door open completed!


# 4. Close the connection

In [7]:
# Close the connection
await tc.stop()
print("✓ Connection closed.")

✓ Connection closed.
